In [3]:
import tensorflow as tf
import keras
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
# from model.configurations import WINDOW_SIZE, N_CHANNELS
from training.train import load_tfrecord_dataset


model = "/home/phatdat/Desktop/Sleep-Apnea-Detection/model/checkpoints/best_params_10epochs.keras"
model = keras.models.load_model(model)

2024-11-28 16:57:17.506130: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-28 16:57:17.513486: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732787837.522516 1078332 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732787837.525104 1078332 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-28 16:57:17.534599: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [4]:
train_set, _ = load_tfrecord_dataset("/mnt/dat/prepped/apnea_sp02_pr/train.tfrecord", batch_size=1024, shuffle=False)
test_set, _ = load_tfrecord_dataset("/mnt/dat/prepped/apnea_sp02_pr/test.tfrecord", batch_size=1024, shuffle=False)

In [5]:
def get_true_pred(model, dataset, batch_size):
    y_pred_prob = model.predict(dataset, batch_size=batch_size)
    y_true = []
    for _, y in dataset:
        y_true.append(y.numpy())
    y_true = np.vstack(y_true)
    return y_true, y_pred_prob

In [6]:
y_train_true, y_train_pred_prob = get_true_pred(model, train_set, batch_size=1024)
y_test_true, y_test_pred_prob = get_true_pred(model, test_set, batch_size=1024)

2024-11-28 16:57:19.147839: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:370] TFRecordDataset `buffer_size` is unspecified, default to 262144
I0000 00:00:1732787839.176721 1078332 cuda_dnn.cc:529] Loaded cuDNN version 90300


1592/1592 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step


2024-11-28 16:57:28.569805: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


     28/Unknown 0s 6ms/step

2024-11-28 16:57:38.183502: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


1820/1820 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step


2024-11-28 16:57:59.759659: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [40]:
def sen_spec(y_true, y_pred_probs, threshold=None):
    """
    Calculate the geometric mean (G-mean) for a given threshold.
    """
    # Convert predicted probabilities to binary predictions based on threshold
    if threshold is None:
        y_pred = y_pred_probs
    else:
        y_pred = (y_pred_probs >= threshold).astype(int)
        
    # Compute confusion matrix
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

    # Compute sensitivity and specificity
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

    return sensitivity, specificity

def optimize_threshold(y_true, y_pred_probs):
    """
    Find the threshold that maximizes the G-mean for binary classification.
    """
    best_threshold = -1
    best_gmean = -1
    thresholds = np.linspace(0, 0.2, 101)

    for threshold in thresholds:
        sensitivity, specificity = sen_spec(y_true, y_pred_probs, threshold)
        gmean = np.sqrt(sensitivity * specificity)

        if gmean > best_gmean:
            best_gmean = gmean
            best_threshold = threshold

    return best_threshold, best_gmean

In [61]:
best_threshold, best_gmean = optimize_threshold(y_train_true.flatten(), y_train_pred_prob.flatten())
best_threshold, best_gmean

(np.float64(0.02), np.float64(0.8701193512121673))

In [65]:
y_test_pred = (y_test_pred_prob >= 0.1).astype(int)

In [64]:
print(classification_report(y_test_true, y_test_pred))

              precision    recall  f1-score   support

           0       0.99      0.90      0.94   1805216
           1       0.19      0.74      0.31     57852

    accuracy                           0.90   1863068
   macro avg       0.59      0.82      0.62   1863068
weighted avg       0.97      0.90      0.92   1863068



In [66]:
sensitivity, specificity = sen_spec(y_test_true, y_test_pred)
sensitivity, specificity

(np.float64(0.5164212127497753), np.float64(0.9728137796252637))

In [1]:
train_path = "alsdjflaoinog/asdfjowinfas/jakjho/train_abc.tfrecord"


mid_name = train_path[train_path.rfind("/") + 1:]
mid_name = mid_name[len("train"):-len("tf.record")]
mid_name

'_abc'